In [123]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input, Embedding, Dropout, LayerNormalization
from tensorflow.keras.models import Model
import numpy as np
import os
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

In [124]:
encoding_legend = {
    1: 'MRI_CCS_11', 2: 'MRI_EXU_95', 3: 'MRI_FRR_18', 4: 'MRI_FRR_257',
    5: 'MRI_FRR_264', 6: 'MRI_FRR_3', 7: 'MRI_FRR_34', 8: 'MRI_MPT_1005',
    9: 'MRI_MSR_100', 10: 'MRI_MSR_104', 11: 'MRI_MSR_21', 12: 'MRI_MSR_34',
    0: 'PADDED',  # Add a padding category
    10: 'START',  # Start token
    9: 'END'      # End token
}

In [125]:
# Positional Encoding
def positional_encoding(position, d_model):
    angle_rads = np.arange(position)[:, np.newaxis] / np.power(10000, (2 * (np.arange(d_model) // 2)) / np.float32(d_model))
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    return tf.cast(angle_rads[np.newaxis, ...], dtype=tf.float32)

In [126]:
# Define MultiHeadAttention Layer
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        assert d_model % num_heads == 0
        self.depth = d_model // num_heads
        self.wq = Dense(d_model)
        self.wk = Dense(d_model)
        self.wv = Dense(d_model)
        self.dense = Dense(d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]
        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)
        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)
        attention, _ = self.scaled_dot_product_attention(q, k, v, mask)
        attention = tf.transpose(attention, perm=[0, 2, 1, 3])
        attention = tf.reshape(attention, (batch_size, -1, self.d_model))
        return self.dense(attention)

    def scaled_dot_product_attention(self, q, k, v, mask):
        matmul_qk = tf.matmul(q, k, transpose_b=True)
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
        if mask is not None:
            scaled_attention_logits += (mask * -1e9)
        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
        output = tf.matmul(attention_weights, v)
        return output, attention_weights

In [127]:
# Define Positionwise Feedforward Layer
class PositionwiseFeedforward(tf.keras.layers.Layer):
    def __init__(self, d_model, dff):
        super(PositionwiseFeedforward, self).__init__()
        self.d_model = d_model
        self.dff = dff
        self.dense1 = Dense(dff, activation='relu')
        self.dense2 = Dense(d_model)

    def call(self, x):
        x = self.dense1(x)
        x = self.dense2(x)
        return x

# Define Transformer Block
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, dropout_rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(d_model, num_heads)
        self.ffn = PositionwiseFeedforward(d_model, dff)
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(dropout_rate)
        self.dropout2 = Dropout(dropout_rate)

    def call(self, x, training, mask):
        attn_output = self.att(x, x, x, mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)
        return out2

# Define Encoder Layer
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, dropout_rate=0.1):
        super(Encoder, self).__init__()
        self.d_model = d_model
        self.num_layers = num_layers
        self.embedding = Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)
        self.dropout = Dropout(dropout_rate)
        self.enc_layers = [TransformerBlock(d_model, num_heads, dff, dropout_rate) for _ in range(num_layers)]

    def call(self, x, training, mask):
        seq_len = tf.shape(x)[1]
        x = self.embedding(x)
        x += self.pos_encoding[:, :seq_len, :]
        x = self.dropout(x, training=training)
        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)
        return x

# Define Decoder Layer
class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, maximum_position_encoding, dropout_rate=0.1):
        super(Decoder, self).__init__()
        self.d_model = d_model
        self.num_layers = num_layers
        self.embedding = Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)
        self.dropout = Dropout(dropout_rate)
        self.dec_layers = [TransformerBlock(d_model, num_heads, dff, dropout_rate) for _ in range(num_layers)]

    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        seq_len = tf.shape(x)[1]
        x = self.embedding(x)
        x += self.pos_encoding[:, :seq_len, :]
        x = self.dropout(x, training=training)
        for i in range(self.num_layers):
            x = self.dec_layers[i](x, enc_output, training, look_ahead_mask)
        return x

In [128]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, dropout_rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(d_model, num_heads)
        self.ffn = PositionwiseFeedforward(d_model, dff)
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(dropout_rate)
        self.dropout2 = Dropout(dropout_rate)

    def call(self, x, training=None, mask=None):  # Ensure training, mask passed as keywords
        attn_output = self.att(x, x, x, mask)  # x already contains embeddings
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)
        return out2


In [129]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, dropout_rate=0.1):
        super(Encoder, self).__init__()
        self.d_model = d_model
        self.num_layers = num_layers
        self.embedding = Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)
        self.dropout = Dropout(dropout_rate)
        self.enc_layers = [TransformerBlock(d_model, num_heads, dff, dropout_rate) for _ in range(num_layers)]

    def call(self, x, training=None, mask=None):  # training and mask as keyword arguments
        seq_len = tf.shape(x)[1]
        x = self.embedding(x)
        x += self.pos_encoding[:, :seq_len, :]
        x = self.dropout(x, training=training)
        
        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training=training, mask=mask)
        
        return x

In [130]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, maximum_position_encoding, dropout_rate=0.1):
        super(Decoder, self).__init__()
        self.d_model = d_model
        self.num_layers = num_layers
        self.embedding = Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)
        self.dropout = Dropout(dropout_rate)
        self.dec_layers = [TransformerBlock(d_model, num_heads, dff, dropout_rate) for _ in range(num_layers)]

    def call(self, x, enc_output, training=None, look_ahead_mask=None, padding_mask=None):  # Ensure keyword args
        seq_len = tf.shape(x)[1]
        x = self.embedding(x)
        x += self.pos_encoding[:, :seq_len, :]
        x = self.dropout(x, training=training)
        
        for i in range(self.num_layers):
            x = self.dec_layers[i](x, training=training, mask=look_ahead_mask)
        
        return x

In [131]:
class Transformer(Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, pe_input, pe_target, rate=0.1):
        super(Transformer, self).__init__()
        self.encoder = Encoder(num_layers, d_model, num_heads, dff, input_vocab_size, pe_input, rate)
        self.decoder = Decoder(num_layers, d_model, num_heads, dff, target_vocab_size, pe_target, rate)
        self.final_layer = Dense(target_vocab_size)

    def call(self, inputs, targets, training=None, look_ahead_mask=None, padding_mask=None):
        enc_output = self.encoder(inputs, training=training, mask=padding_mask)
        dec_output = self.decoder(targets, enc_output, training=training, look_ahead_mask=look_ahead_mask, padding_mask=padding_mask)
        final_output = self.final_layer(dec_output)
        return final_output


In [132]:
# Load and preprocess data
def preprocess_data(directory, encoding_legend):
    all_blocks = []
    all_source_ids = np.array(list(encoding_legend.keys())).reshape(-1, 1)
    encoder = OneHotEncoder(sparse=False)
    encoder.fit(all_source_ids)

    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            file_path = os.path.join(directory, filename)
            block = pd.read_csv(file_path)

            assert block.shape == (36, 5), f"Block {filename} has an unexpected shape {block.shape}"

            source_ids = block[['sourceID']].values.reshape(-1, 1)
            one_hot_encoded_sourceID = encoder.transform(source_ids)

            # Normalize additional features
            timediff = block[['timediff']].values
            ptab = np.nan_to_num(block[['PTAB']].values)
            timediff = (timediff - np.mean(timediff)) / np.std(timediff)
            ptab = (ptab - np.mean(ptab)) / np.std(ptab)
            body_group_from = block[['BodyGroup_from']].values
            body_group_to = block[['BodyGroup_to']].values

            # Create input block with start and end tokens
            X_block = np.concatenate((one_hot_encoded_sourceID, timediff, ptab, body_group_from, body_group_to), axis=1)
            all_blocks.append(X_block)

    all_blocks = np.stack(all_blocks, axis=0)
    return all_blocks, encoder

In [133]:
# Directory containing the 150 CSV data blocks
data_directory = "../data/filtered_blocks_padded/"

In [134]:
# Preprocess data and obtain one-hot encoding for sourceIDs
all_blocks, encoder = preprocess_data(data_directory, encoding_legend)

# Parameters for model
num_layers = 4
d_model = 128
num_heads = 8
dff = 512
input_vocab_size = len(encoder.categories_[0])  # Number of unique sourceIDs
target_vocab_size = input_vocab_size  # Assuming prediction is similar to input
pe_input = 1000
pe_target = 1000
dropout_rate = 0.1

C:\Users\z004uyxr\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [146]:
# Create the Transformer model
transformer = Transformer(num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, pe_input, pe_target, dropout_rate)


In [147]:
# Example Input (creating dummy data for testing)
# Adjust batch size based on your data length
batch_size = min(32, len(df) // 2)  # Ensure at least some data for batching
max_seq_length = 36  # Adjust as necessary

In [148]:
# Generate dummy input and target sequences (for illustration)
inputs = tf.random.uniform((batch_size, max_seq_length), dtype=tf.int64, minval=0, maxval=input_vocab_size)
targets = tf.random.uniform((batch_size, max_seq_length), dtype=tf.int64, minval=0, maxval=target_vocab_size)
look_ahead_mask = None
padding_mask = None

In [149]:
# Forward Pass
output = transformer(inputs=inputs, targets=targets, training=True, look_ahead_mask=look_ahead_mask, padding_mask=padding_mask)
print(output.shape)  # Should now work correctly


(18, 36, 13)


In [150]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))  # Mask out padding tokens
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)


In [151]:
import os
import pandas as pd

# Define the path to the directory containing the data blocks
data_directory = "../data/filtered_blocks_padded/"

# Initialize an empty list to hold all DataFrames
data_frames = []

for file_name in os.listdir(data_directory):
    if file_name.endswith(".csv"):
        file_path = os.path.join(data_directory, file_name)
        df = pd.read_csv(file_path)
        print(f"Loaded {file_name}, shape: {df.shape}")

        
        # Append the DataFrame to the list
        data_frames.append(df)

# Concatenate all DataFrames into one DataFrame
combined_data = pd.concat(data_frames, ignore_index=True)

# Check the first few rows of the combined data
print(combined_data.head())

# Ensure the DataFrame is loaded properly
print(df.head())
print(f"DataFrame size: {df.shape}")



Loaded padded_block_1.csv, shape: (36, 5)
Loaded padded_block_10.csv, shape: (36, 5)
Loaded padded_block_100.csv, shape: (36, 5)
Loaded padded_block_101.csv, shape: (36, 5)
Loaded padded_block_102.csv, shape: (36, 5)
Loaded padded_block_103.csv, shape: (36, 5)
Loaded padded_block_104.csv, shape: (36, 5)
Loaded padded_block_105.csv, shape: (36, 5)
Loaded padded_block_106.csv, shape: (36, 5)
Loaded padded_block_107.csv, shape: (36, 5)
Loaded padded_block_108.csv, shape: (36, 5)
Loaded padded_block_109.csv, shape: (36, 5)
Loaded padded_block_11.csv, shape: (36, 5)
Loaded padded_block_110.csv, shape: (36, 5)
Loaded padded_block_111.csv, shape: (36, 5)
Loaded padded_block_112.csv, shape: (36, 5)
Loaded padded_block_113.csv, shape: (36, 5)
Loaded padded_block_114.csv, shape: (36, 5)
Loaded padded_block_115.csv, shape: (36, 5)
Loaded padded_block_116.csv, shape: (36, 5)
Loaded padded_block_117.csv, shape: (36, 5)
Loaded padded_block_118.csv, shape: (36, 5)
Loaded padded_block_119.csv, shape: 

In [152]:
# Adjusting the preprocessing function
def preprocess_data(df, sequence_length, step_size=1):
    inputs = df['sourceID'].values
    targets = df['sourceID'].shift(-1).fillna(0).values  # Shift by 1 and replace NaN with 0

    # Convert to integer type for model input
    inputs = inputs.astype(int)
    targets = targets.astype(int)

    input_sequences = []
    target_sequences = []

    # Slide over the entire dataset with overlapping windows
    for i in range(0, len(inputs) - sequence_length + 1, step_size):  # Added step_size for overlap
        input_sequences.append(inputs[i:i + sequence_length])
        target_sequences.append(targets[i:i + sequence_length])

    inputs = np.array(input_sequences)
    targets = np.array(target_sequences)

    return inputs, targets

sequence_length = 36  # Example sequence length
inputs, targets = preprocess_data(df, sequence_length)
print(f"Inputs shape: {inputs.shape}, Targets shape: {targets.shape}")

# Create a dataset from the inputs and targets
dataset = tf.data.Dataset.from_tensor_slices((inputs, targets))

# Create a dataset with a proper batch size
dataset = dataset.batch(batch_size, drop_remainder=False)

# Print the dataset size to ensure it's not empty
print(f"Dataset size: {len(dataset)}")

# Check inputs and targets before reshaping
inputs, targets = preprocess_data(df, sequence_length)
print(f"Inputs shape: {inputs.shape}, Targets shape: {targets.shape}")

if inputs.size > 0 and targets.size > 0:
    dataset = tf.data.Dataset.from_tensor_slices((inputs, targets))
    dataset = dataset.batch(batch_size, drop_remainder=False)
    print(f"Dataset size: {len(dataset)}")
else:
    print("No sequences generated. Dataset size is 0.")



# Define loss and optimizer
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

@tf.function
def train_step(batch_inputs, batch_targets):
    with tf.GradientTape() as tape:
        predictions = transformer(batch_inputs, batch_targets, training=True, look_ahead_mask=None, padding_mask=None)
        loss = loss_function(batch_targets, predictions)  # Ensure loss_function is defined
    gradients = tape.gradient(loss, transformer.trainable_variables)
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))
    
    return loss  # Make sure this returns the loss
    print(f"Loss: {loss.numpy()}")
    print(f"Inputs shape: {batch_inputs.shape}, Targets shape: {batch_targets.shape}")

Inputs shape: (1, 36), Targets shape: (1, 36)
Dataset size: 1
Inputs shape: (1, 36), Targets shape: (1, 36)
Dataset size: 1


In [153]:
transformer.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=loss_function
)

In [154]:
num_epochs = 100

for epoch in range(num_epochs):
    epoch_loss = 0
    for batch_inputs, batch_targets in dataset:
        print(f"Inputs shape: {batch_inputs.shape}, Targets shape: {batch_targets.shape}")
        
        # Ensure the shapes are correct for your model
        batch_inputs = tf.reshape(batch_inputs, [batch_size, -1])
        batch_targets = tf.reshape(batch_targets, [batch_size, -1])

        # Get the loss from the train_step function
        loss = train_step(batch_inputs, batch_targets)
        epoch_loss += loss.numpy()

    # Only print loss if dataset is not empty
    if len(dataset) > 0:
        print(f"Epoch {epoch + 1}, Loss: {epoch_loss / len(dataset)}")
    else:
        print(f"Epoch {epoch + 1}, No batches to train on.")


Inputs shape: (1, 36), Targets shape: (1, 36)


C:\Users\z004uyxr\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\optimizers\base_optimizer.py:731: UserWarning: Gradients do not exist for variables ['embeddings', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'gamma', 'beta', 'gamma', 'beta', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'gamma', 'beta', 'gamma', 'beta', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'gamma', 'beta', 'gamma', 'beta', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'gamma', 'beta', 'gamma', 'beta'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


Epoch 1, Loss: 1.7621448040008545
Inputs shape: (1, 36), Targets shape: (1, 36)
Epoch 2, Loss: 0.9441693425178528
Inputs shape: (1, 36), Targets shape: (1, 36)
Epoch 3, Loss: 0.8866862654685974
Inputs shape: (1, 36), Targets shape: (1, 36)
Epoch 4, Loss: 0.8138106465339661
Inputs shape: (1, 36), Targets shape: (1, 36)
Epoch 5, Loss: 0.7934479117393494
Inputs shape: (1, 36), Targets shape: (1, 36)
Epoch 6, Loss: 0.7764068245887756
Inputs shape: (1, 36), Targets shape: (1, 36)
Epoch 7, Loss: 0.7428820133209229
Inputs shape: (1, 36), Targets shape: (1, 36)
Epoch 8, Loss: 0.742789626121521
Inputs shape: (1, 36), Targets shape: (1, 36)
Epoch 9, Loss: 0.7376158237457275
Inputs shape: (1, 36), Targets shape: (1, 36)
Epoch 10, Loss: 0.7443320751190186
Inputs shape: (1, 36), Targets shape: (1, 36)
Epoch 11, Loss: 0.6809525489807129
Inputs shape: (1, 36), Targets shape: (1, 36)
Epoch 12, Loss: 0.7604783177375793
Inputs shape: (1, 36), Targets shape: (1, 36)
Epoch 13, Loss: 0.7235222458839417
Inp

In [155]:
# Sample input, ensure it matches the expected shape (batch_size, sequence_length)
sample_inputs = np.array([[10, 4, 5, 1, 12, 8, 9]])  
sample_inputs = tf.convert_to_tensor(sample_inputs)

# Pad the inputs to match the expected input length of 36
padded_inputs = tf.keras.preprocessing.sequence.pad_sequences(sample_inputs, maxlen=36, padding='post')

# Reshape padded inputs for batch size of 1
padded_inputs = tf.expand_dims(padded_inputs, axis=0)  # Now shape (1, 36)

# Make predictions
predictions = transformer(padded_inputs, targets=tf.zeros((1, 36)), training=False, look_ahead_mask=None, padding_mask=None)

# Convert predictions from logits to class indices
predicted_indices = tf.argmax(predictions, axis=-1).numpy()

# Print the predicted sequences
for i, seq in enumerate(predicted_indices):
    print(f"Predicted sequence {i + 1}: {seq}")

# Remove padding (optional)
predicted_sequence = np.where(seq == 0, -1, seq)  # Replace 0 (PADDED) with -1 or another value
print(f"Final predicted sequence (no padding): {predicted_sequence}")
print("Raw predictions:", predictions.numpy())



Predicted sequence 1: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
Final predicted sequence (no padding): [-1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1
 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1]
Raw predictions: [[[11.141112    0.7925508  -2.2449324  -1.9368104  -0.66093343
   -0.42378095 -0.32271868  1.0319393   0.34296688 -0.34836242
    0.60909474 -4.1248612   0.3573203 ]
  [11.141213    0.7959757  -2.2479897  -1.9368912  -0.66069156
   -0.42973408 -0.3171384   1.0291585   0.3443183  -0.34689116
    0.6109291  -4.127986    0.35510483]
  [11.139138    0.79992646 -2.2505004  -1.9388428  -0.6607887
   -0.43323648 -0.31596297  1.0266678   0.34638086 -0.34579125
    0.6135734  -4.1344137   0.35545665]
  [11.136744    0.80324954 -2.2512136  -1.9425532  -0.659811
   -0.43253082 -0.3188786   1.0250657   0.348593   -0.3459526
    0.6155815  -4.139879    0.35716778]
  [11.137002    0.80491996 -2.2501664  -1.9466611  -0.6588898
   -0.42769122 

In [156]:
transformer.summary()

Model: "transformer_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ encoder_4 (Encoder)                  │ ?                           │         794,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ decoder_4 (Decoder)                  │ ?                           │         794,752 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_244 (Dense)                    │ (18, 36, 13)                │           1,677 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,591,181 (6.07 MB)

 Trainable params: 1,591,181 (6.07 MB)

 Non-trainable params: 0 (0.00 B)